# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [1]:
import requests
import time
import re

from random import randint

In [2]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [3]:
import time


def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """

    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'{func.__name__} потребовалось {end - start:.6f} секунд')
        return result

    return wrapper


## Задание 2

In [4]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """

    def wrapper(*args, **kwargs):
        arg_str = ', '.join(repr(arg) for arg in args)
        if arg_str != '':
            print(f'Аргументы функции {func.__name__}: {arg_str}')
        else:
            print(f'Функции {func.__name__} вызвана без аргументов')
        result = func(*args, **kwargs)
        return result
    return wrapper



## Задание 3

In [5]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """

    def wrapper(*args, **kwargs):
        wrapper.calls += 1
        print(f"Функция '{func.__name__}' вызвана {wrapper.calls} раз.")
        return func(*args, **kwargs), wrapper.calls

    wrapper.calls = 0
    return wrapper


## Задание 4

In [21]:
from functools import wraps
import typing


def memo(func):
    """
    Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
    """

    @wraps(func)
    def fmemo(*args):
        if all(isinstance(arg, typing.Hashable) for arg in args):
            if args in fmemo.cache:
                result = fmemo.cache[args]
            else:
                result = fmemo.cache[args] = func(*args)
            return result
        else:
            return func(*args)

    fmemo.cache = {}
    return fmemo


## Тестирование

In [22]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Функция 'wrapper' вызвана 1 раз.
Аргументы функции wrapper: 'whole'
word_count потребовалось 0.931126 секунд
('Cлово whole встречается 176 раз', 1)


In [38]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [43]:
# измеряем время выполнения
@benchmark
def call():
  fib(30)
call()

call потребовалось 0.348126 секунд


In [44]:
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [45]:
# измеряем время выполнения
@benchmark
def call():
  fib(30)
call()

call потребовалось 0.000261 секунд
